<a href="https://colab.research.google.com/github/daniel-men/transformerNotebookColab/blob/main/torch_io_tutorial_for_transf3d_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## self-attention-cv : illustration of a training process with subvolume sampling for 3d segmentation

The dataset can be found here: https://iseg2019.web.unc.edu/ . i uploaded it and mounted from my gdrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
import tarfile
root_path = '/gdrive/My Drive/Task01_BrainTumour.tar' 
!echo "Download and extracting folders..."
zip_ref = tarfile.TarFile(root_path, 'r')
zip_ref.extractall("./")
zip_ref.close()
!echo "Finished"
!pip install torchio
!pip install self-attention-cv

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Download and extracting folders...
Finished
     |████████████████████████████████| 143kB 19.0MB/s 
     |████████████████████████████████| 42.5MB 63kB/s 
     |████████████████████████████████| 286kB 18.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [ ]:
!ls ./Task01_BrainTumour
import json
with open("./Task01_BrainTumour/dataset.json", 'r') as f:
  json_data = json.load(f)
print(json_data)

dataset.json  imagesTr	imagesTs  labelsTr
{'name': 'BRATS', 'description': 'Gliomas segmentation tumour and oedema in on brain images', 'reference': 'https://www.med.upenn.edu/sbia/brats2017.html', 'licence': 'CC-BY-SA 4.0', 'release': '2.0 04/05/2018', 'tensorImageSize': '4D', 'modality': {'0': 'FLAIR', '1': 'T1w', '2': 't1gd', '3': 'T2w'}, 'labels': {'0': 'background', '1': 'edema', '2': 'non-enhancing tumor', '3': 'enhancing tumour'}, 'numTraining': 484, 'numTest': 266, 'training': [{'image': './imagesTr/BRATS_457.nii.gz', 'label': './labelsTr/BRATS_457.nii.gz'}, {'image': './imagesTr/BRATS_306.nii.gz', 'label': './labelsTr/BRATS_306.nii.gz'}, {'image': './imagesTr/BRATS_206.nii.gz', 'label': './labelsTr/BRATS_206.nii.gz'}, {'image': './imagesTr/BRATS_449.nii.gz', 'label': './labelsTr/BRATS_449.nii.gz'}, {'image': './imagesTr/BRATS_318.nii.gz', 'label': './labelsTr/BRATS_318.nii.gz'}, {'image': './imagesTr/BRATS_218.nii.gz', 'label': './labelsTr/BRATS_218.nii.gz'}, {'image': './imag

## Training example

In [1]:
import glob
import torchio as tio
import torch
from torch.utils.data import DataLoader

paths_t1 = sorted(glob.glob('./Task01_BrainTumour/imagesTr/*.nii.gz'))
#paths_t2 = sorted(glob.glob('./iSeg-2019-Training/*T2.img'))
paths_seg = sorted(glob.glob('./Task01_BrainTumour/labelsTr/*.nii.gz'))
assert len(paths_t1) == len(paths_seg)

subject_list = []
for pat in zip(paths_t1, paths_seg):
  path_t1, path_seg = pat
  image = tio.ScalarImage(path_t1, channels_last=True)
  #t1 = tio.ScalarImage(tensor=image.data[:, :, :, 0].unsqueeze(0))
  #t2 = tio.ScalarImage(tensor=image.data[:, :, :, 2].unsqueeze(0))
  subject = tio.Subject(image=image, label=tio.LabelMap(path_seg)) 
  subject_list.append(subject)

transforms = [tio.RescaleIntensity((0, 1)),tio.RandomAffine() ]
transform = tio.Compose(transforms)

subjects_dataset = tio.SubjectsDataset(subject_list, transform=transform)

patch_size = 24
queue_length = 300
samples_per_volume = 50
sampler = tio.data.UniformSampler(patch_size)

patches_queue = tio.Queue(
subjects_dataset,
queue_length,
samples_per_volume,sampler, num_workers=1)

patches_loader = DataLoader(patches_queue, batch_size=16)

ModuleNotFoundError: ignored

In [ ]:
from tqdm.notebook import tqdm
from self_attention_cv.Transformer3Dsegmentation import Transformer3dSeg

def crop_target(img, target_size):
  dim = img.shape[-1]
  center = dim//2
  start_dim = center - (target_size//2) - 1
  end_dim = center + (target_size//2)
  return img[:,0,start_dim:end_dim,start_dim:end_dim,start_dim:end_dim].long()

target_size = 3 # as in the paper 
patch_dim = 8
num_epochs = 3
num_classes = 4
model = Transformer3dSeg(subvol_dim=patch_size, patch_dim=patch_dim,
                         in_channels=4, blocks=2, num_classes=num_classes).cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(len(patches_loader))
for epoch_index in range(num_epochs):
  epoch_loss = 0
  for c,patches_batch in tqdm(enumerate(patches_loader)):
    optimizer.zero_grad()
    
    input_t1 = patches_batch['image'][tio.DATA]  
    #input_t2 = patches_batch['t2'][tio.DATA]

    #input_tensor = torch.cat([input_t1, input_t2], dim=1).cuda()
    input_tensor = input_t1.cuda()
    
    logits = model(input_tensor) # 8x8x8 the 3d transformer-based approach

    # for the 3d transformer-based approach the target must be cropped again to the desired size
    targets = patches_batch['label'][tio.DATA]  
    cropped_target = crop_target(targets, target_size).cuda()

    loss = criterion(logits, cropped_target)
    loss.backward()
    optimizer.step()
    epoch_loss = epoch_loss+loss.cpu().item()

  print(f'epoch {epoch_index} loss {epoch_loss/c}')
    



1513



epoch 0 loss 0.09744605868093906



epoch 1 loss 0.0708814951198215



epoch 2 loss 0.06348189538988105



epoch 3 loss 0.06510048643183104



epoch 4 loss 0.0656701507714436



epoch 5 loss 0.05592624643509105



epoch 6 loss 0.06275332329900862


## Inference

In [ ]:
import torch
import torch.nn as nn
import torchio as tio
patch_overlap = 0
patch_size = 24, 24, 24
target_patch_size = 3

#input sampling
grid_sampler = tio.inference.GridSampler(subject_list[0], patch_size, patch_overlap)
patch_loader = torch.utils.data.DataLoader(grid_sampler, batch_size=4)
# target vol sampling
grid_sampler_target = tio.inference.GridSampler(subject_list[0], target_patch_size, patch_overlap)
aggregator = tio.inference.GridAggregator(grid_sampler_target)
target_loader = torch.utils.data.DataLoader(grid_sampler_target, batch_size=4)

model.eval()

print(tio.LOCATION)

with torch.no_grad():
  for patches_batch,target_patches in zip(patch_loader,target_loader):

    #input_t1 = patches_batch['t1'][tio.DATA]  
    #input_t2 = patches_batch['t2'][tio.DATA]
    #input_tensor = torch.cat([input_t1, input_t2], dim=1).float().cuda()

    input_tensor = patches_batch['image'][tio.DATA].cuda()

    locations = target_patches[tio.LOCATION]
    logits = model(input_tensor)
    labels = logits.argmax(dim=tio.CHANNELS_DIMENSION, keepdim=True)
    outputs = labels
    aggregator.add_batch(outputs.type(torch.int32), locations)

  print('output tensor shape:',outputs.shape)
  output_tensor = aggregator.get_output_tensor()
  output_tensor.reshape(1, 240, 240, 155)
  out_img = tio.ScalarImage(tensor=output_tensor)
  out_img.show()
  print(output_tensor.shape)

output tensor shape: torch.Size([4, 1, 3, 3, 3])
torch.Size([1, 240, 240, 155])
